<a href="https://colab.research.google.com/github/stepyt/vs_cylinder_ann_pso/blob/master/PSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **OPTIMIZATION OF VARIABLE STIFFNESS CYLINDRICAL SHELL**

In this notebook the optimized neural network will be used for the optimization of the fibers path parameters in order to achieve the maximum buckling value. To this end a bio-inspired metaheuristic algorithm, named PSO, will be used. Below a picture of the idea behind the code.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import sys

We need to import the normalization variables in order to normalize the particles position and recover the output of the networks. These normalization variables must be the same used during the training of the network.

In [3]:
info = pd.read_csv('../metamodel/data367/model_info.csv', sep=",")
info.index = ['Value']
eff_plies = int(info['EffectivePlies'].values)
train_smp = int(info['Train'].values)
info.head()

,Height,Radius,MaxCurvature,MeshSize,Plies,EffectivePlies,Symmetric,Balanced,AnglesFunction,Train,Test
Value,705,300,0.001575,5,8,4,True,False,harmlin,367,92


The steps below are the same on the notebook of the nn.

In [5]:
data_orig = pd.read_csv('../metamodel/data367/data.csv', sep=',')
data = data_orig.drop(columns='Stiffness')

In [6]:
def range_norm(x):
  """ Normalize x in range [0, 1] """
  x_min = np.min(x, axis=0)
  x_max = np.max(x, axis=0)
  x_norm = (x - x_min) / (x_max - x_min)
  return x_norm, x_min, x_max

def std_norm(x, m=False, s=False):
  """ Remove mean and fix standard deviation to 1 """
  if isinstance(m, (np.ndarray, np.generic))==False and isinstance(m, (np.ndarray, np.generic))==False:
    m = np.mean(x, axis=0)
    s = np.std(x, axis=0)
  x_norm = (x - m) / s
  return x_norm, m, s

def inv_range_norm(x_norm, x_min, x_max):
  """ Recover the original value of the variables """
  x = x_norm * (x_max - x_min) + x_min
  return x

def inv_std_norm(x_norm, m, s):
  """ Recover the original value of the variables """
  x = x_norm * s + m
  return x

In [7]:
X = data_orig.drop(columns='Buckling').values
Y = data_orig['Buckling'].values

X_orig_tv = X[:train_smp, :]
Y_orig_tv = Y[:train_smp]

In [8]:
_, m_data, s_data = std_norm(X_orig_tv)
_, m_target, s_target = std_norm(Y_orig_tv)

Now we have to recover the boundaries used during the design of experiments in order to be able to define the deisign space in which search for an optimum configuration.

In [28]:
boundaries = pd.read_csv('../metamodel/data367/features_min_max.csv', sep=',')
ds = {}
out = ['RUN']
for ply in range(eff_plies):
    for key in boundaries.keys():
        ds[key.upper() + str(ply+1)] = boundaries[key]
        out += [key.upper() + str(ply+1)]
design_space = pd.DataFrame(ds)

## **Particle Swarm Optimizer**

Now we can define some parameters of the optimizer:

In [0]:
particles = 80
max_iter = 1

In [0]:
tic = time.time()
for run in range(max_iter):

    print(run)

    pos, vel = particles_init(particles, boundaries)
    f_bck, k_bck = net_prediction(pos, x_min_max, y_min_max)
    score = 1 / f_bck
    p_best = copy(score)
    pos_p_best = copy(pos)
    part_best = np.argmin(score)
    g_best = score[part_best]
    pos_g_best = pos[part_best, :]
    stiffness_best = 0
    buckling_best = 0
    history_buckling_best = []

    for k in range(0, max_iter):

        new_pos = fly(pos, vel, pos_p_best, pos_g_best, boundaries, max_iter, k + 1)
        f_bck, k_bck = net_prediction(new_pos, x_min_max, y_min_max)
        score = 1 / f_bck

        pos_p_best_update = np.where(p_best > score)
        p_best[pos_p_best_update] = score[pos_p_best_update]
        pos_p_best[pos_p_best_update, :] = new_pos[pos_p_best_update, :]

        part_best = np.argmin(score)
        if g_best > score[part_best]:
            g_best = score[part_best]
            pos_g_best = new_pos[part_best, :]
            buckling_best = f_bck[part_best]
            stiffness_best = k_bck[part_best]

        history_buckling_best.append(buckling_best)

        print(stiffness_best)
        print(buckling_best)

    outputFile.write(str(run+1) + ', ' +
                     str(buckling_best) + ', ' + str(stiffness_best) + ', ' +
                     str(pos_g_best[0]) + ', ' + str(pos_g_best[1]) + ', ' +
                     str(pos_g_best[2]) + ', ' + str(pos_g_best[3]) + ', ' +
                     str(pos_g_best[4]) + ', ' + str(pos_g_best[5]) + ', ' +
                     str(pos_g_best[6]) + ', ' + str(pos_g_best[7]) + '\n')

    history = np.array([history_buckling_best])
    name_history = 'history_best_' + str(run + 1) + '.csv'
    np.savetxt(name_history, history, delimiter=",")
toc = time.time()
train_time = toc - tic
print('TIME: ' + str(train_time))
outputFile.close()